In [143]:
import pandas as pd
import re
import numpy as np
import pickle
import scipy

In [236]:
def remove_extra(s):
    return(re.sub(r'\([^\d]*\)|\{.*\}|\'|\"', "", s).strip())

In [237]:
#701 actors

In [238]:
def get_non_duplicated(artist_dataframe):
    artist_counts = artist_dataframe[['movies','artist']].groupby('artist',as_index=False).agg(['count'])
    artist_counts.columns = [''.join(col).strip() for col in artist_counts.columns.values]
    artist_counts2 = artist_counts.reset_index()
    print('Only',artist_counts2['artist'].count(),' unique names')
    
    artist_counts2  = artist_counts2.loc[artist_counts2['moviescount'] >1]
    duplicate_artists = list(artist_counts2['artist'].values)
    #print(len(duplicate_artists),' repeated names')
    duplicated_artist_df  = artist_dataframe.loc[artist_dataframe['artist'].isin(duplicate_artists)]
    non_duplicated_artist_df  = artist_dataframe.loc[~artist_dataframe['artist'].isin(duplicate_artists)]
    print(non_duplicated_artist_df['movies'].count(),' non repeated names')
    fixed_duplicated_artist_df = duplicated_artist_df.groupby('artist').agg({'movies':'sum'})
    fixed_duplicated_artist_df['movies'] = fixed_duplicated_artist_df['movies'].apply(lambda x: list(set(x)))
    print(fixed_duplicated_artist_df['movies'].count(),' responsible for duplicates')
    non_duplicated_artist_df.append(fixed_duplicated_artist_df)
    return non_duplicated_artist_df

    

In [239]:
def get_artist_data(file_name):
    artist_list = []
    artist_movies_list = []
    i = 0
    with open(file_name,'rb') as f:
        lines = f.readlines()
        for line in lines:
            i+=1
            if(i%100000==0):
                print(i)
            line= line.decode('utf-8','ignore').encode("utf-8")
            line_split = line.decode().split('\t\t')
            artist = line_split[0]
            movies = []
            if(len(line_split)>1):
                movies = set(map(remove_extra, line_split[1:]))
                movies =[m for m in movies if(len(m))>0]
            artist_list.append(artist)
            artist_movies_list.append(list(movies))     
    print(len(artist_list)," Lines found")
    artist_dataframe = pd.DataFrame(list(zip(artist_list, artist_movies_list)),columns=['artist','movies'])
    return get_non_duplicated(artist_dataframe)
    



In [240]:
non_duplicated_actor_df = get_artist_data('data/actor_movies.txt')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2167653  Lines found
Only 2166817  unique names
2166116  non repeated names
701  responsible for duplicates


In [241]:
non_duplicated_actress_df = get_artist_data('data/actress_movies.txt')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1182813  Lines found
Only 1182404  unique names
1182084  non repeated names
320  responsible for duplicates


In [242]:
non_duplicated_artist_df = get_non_duplicated(non_duplicated_actor_df.append(non_duplicated_actress_df))
non_duplicated_artist_df['count'] = non_duplicated_artist_df['movies'].apply(lambda x: len(x))
non_duplicated_artist_df.head()

Only 3348112  unique names
3348024  non repeated names
88  responsible for duplicates


,artist,movies,count
0,"$, Claw",[],0
1,"$, Homo","[Suuri illusioni (1985), Nykytaiteen museo (19...",2
2,"$, Steve",[],0
3,"$hort, Too","[Beats, Rhymes & Life: The Travels of a Tribe ...",12
4,"$lim, Bee Moe","[For Thy Love 2 (2009), Fatherhood 101 (2013)]",2


In [243]:
non_duplicated_artist_df.count()

artist    3348024
movies    3348024
count     3348024
dtype: int64

In [244]:
final_df = non_duplicated_artist_df.loc[non_duplicated_artist_df['count'] >=10]

In [245]:
pickle.dump(final_df, open('final.pickle','wb'))

In [246]:
import pickle
import pandas as pd
import re
import numpy as np
import pickle
import scipy
#final_df = pickle.load(open('final.pickle','rb'))
artists = final_df['artist'].values
num_artists = len(artists)#len(artists)

In [247]:
movies_list = final_df['movies'].values
import itertools
merged = list(itertools.chain(*movies_list))
movies = list(set(merged)) 
num_movies =len(movies)


In [248]:
movies_list =[set(movies) for movies in movies_list]

In [249]:
artist_id_map = {}

for i in range(len(artists)):
    artist_id_map[artists[i]] = i
    movie_id_map = {}
for i in range(len(movies)):
    movie_id_map[movies[i]] = i

In [251]:
final_df.count()

artist    112993
movies    112993
count     112993
dtype: int64

In [252]:
row_cnt = 0
artist_movie_mat = []
def fill_matrix(row):
    global row_cnt
    row_cnt +=1
    if(row_cnt%10000==0):
        print(row_cnt)
    row = dict(row)
    movies = row['movies']
    artist = row['artist']
    movie_ids = [movie_id_map[movie] for movie in movies]
    artist_row = np.zeros(num_movies)
    artist_row[movie_ids] = 1
    artist_movie_mat.append(artist_row)
   # artist_movie_mat[artist_id_map[artist],:] = artist_row

In [253]:
def get_intersections(e1,e2):
    return (len(set(e1)&e2),len(e1))
vector_get_intersections =np.vectorize(get_intersections)

In [254]:
import os

In [255]:
len(final_df.loc[final_df['artist'] == artists[17028]]['movies'].values[0])

22

In [256]:
artists[0], artists[9756]

('$hort, Too', 'Campbell, Larkin')

In [257]:
final_df.head()

,artist,movies,count
3,"$hort, Too","[Beats, Rhymes & Life: The Travels of a Tribe ...",12
323,2 Live Crew,"[2 Live Crew: Pop the Pussy (1991), 2 Live Cre...",20
482,50 Cent,"[50 Cent: Bulletproof (2005), Call of Duty: Mo...",39
660,"A., Randy","[The Sketches of Matty Mad Man (2013), Sex (20...",19
669,"A., Sergey","[Night of the Dead Man (2014), Otel mertvetsov...",34


In [258]:
if os.path.exists('edge_list.txt'):
    x = input('Path Exist. Want to create a new?y/n')
    if(x=='y'):
        os.remove('edge_list.txt')

Path Exist. Want to create a new?y/ny


In [ ]:
i = 0
for i in range(num_artists-1):
    i+=1
    if(i%1024==0):
        print(i)
    x= vector_get_intersections(movies_list[i+1:],movies_list[i])
    #print(len(x[0]))
    i_len = len(movies_list[i])
 #   print(len(range(i+1,num_artists)))
    neigh = [(j,x[0][j-i-1],x[1][j-i-1]) for j in range(i+1,num_artists) if x[0][j-i-1]>0 and j!=i]
    with open('edge_list.txt','a+')as  ef:
        for n in neigh:
            ef.write("%d\t%d\t%f\n"%(i,n[0],n[1]*1.0/i_len))
            ef.write("%d\t%d\t%f\n"%(n[0],i,n[1]*1.0/n[2]))
        

1024
2048
3072
4096
5120
6144
7168
8192
9216
10240
11264
12288
13312
14336
15360
16384
17408
18432
19456
20480
21504
22528
23552
24576
25600
26624
27648
28672
29696


In [ ]:
pickle.dump(weights,open('weights.pl','wb'))